## **Dependencies**

In [167]:
import pandas as pd
from tqdm.notebook import tqdm

## **Load JSON**

Choose respective json file to transform to data frame
Json be found in /src/response_[ORG]

### Define Params

In [168]:
##########
# PARAMS #
##########

abbreviation = "bmz" # abbreviation of development bank
src_folder = "../../src/"
response_file = src_folder + f"response_{abbreviation}.json" # json format -> fetched in iati.ipynb
output_file = src_folder + f"transformed_{abbreviation}.csv"

In [169]:
#Read in data into df from json
df = pd.read_json(response_file)
df.head(10)

,sector_code,iati_identifier,title_narrative,document_link_url,reporting_org_ref,sector_vocabulary,activity_date_type,activity_status_code,description_narrative,last_updated_datetime,activity_date_iso_date,recipient_country_code,location_name_narrative,reporting_org_narrative,title_narrative_xml_lang,description_narrative_xml_lang,contact_info_organisation_narrative,recipient_region_code
0,[12230],DE-1-201629492-0,[Basisgesundheitsdienste zur Senkung der Mütte...,"[https://www.bmz.de/de/laender/suedsudan, http...",DE-1,[1],"[1, 2, 3, 4]",3,[Basisgesundheitsdienste zur Senkung der Mütte...,2024-01-22T00:00:00Z,"[2016-09-01T00:00:00Z, 2016-09-01T00:00:00Z, 2...",[SS],[Juba],[Bundesministerium für wirtschaftliche Zusamme...,"[de, en]","[de, en, de, en]",[Bundesministerium für wirtschaftliche Zusamme...,NaN
1,[43040],DE-1-201673854-0,[Nachhaltige Wasserversorgung und Lebensunterh...,"[https://www.bmz.de/de/laender/suedsudan, http...",DE-1,[1],"[1, 2, 3, 4]",3,[Nachhaltige Wasserversorgung und Lebensunterh...,2024-01-22T00:00:00Z,"[2016-04-01T00:00:00Z, 2016-04-01T00:00:00Z, 2...",[SS],[Juba],[Bundesministerium für wirtschaftliche Zusamme...,"[de, en]","[de, en, de, en]",[Bundesministerium für wirtschaftliche Zusamme...,NaN
2,[11330],DE-1-201676782-0,[Wirtschaftliche Selbständigkeit junger Frauen...,"[https://www.bmz.de/de/laender/suedsudan, http...",DE-1,[1],"[1, 2, 3, 4]",3,[Wirtschaftliche Selbständigkeit junger Frauen...,2024-01-22T00:00:00Z,"[2017-01-01T00:00:00Z, 2017-01-01T00:00:00Z, 2...",[SS],[Juba],[Bundesministerium für wirtschaftliche Zusamme...,"[de, en]","[de, en, de, en]",[Bundesministerium für wirtschaftliche Zusamme...,NaN
3,[15220],DE-1-201676790-0,[Stärkung des Gemeindelebens und Friedensbildu...,"[https://www.bmz.de/de/laender/suedsudan, http...",DE-1,[1],"[1, 2, 3, 4]",3,[Stärkung des Gemeindelebens und Friedensbildu...,2024-01-22T00:00:00Z,"[2017-01-01T00:00:00Z, 2017-01-01T00:00:00Z, 2...",[SS],[Juba],[Bundesministerium für wirtschaftliche Zusamme...,"[de, en]","[de, en, de, en]",[Bundesministerium für wirtschaftliche Zusamme...,NaN
4,[11130],DE-1-201676808-0,"[Primarschullehrerausbildung Südsudan, Fortfüh...","[https://www.bmz.de/de/laender/suedsudan, http...",DE-1,[1],"[1, 2, 3, 4]",3,"[Primarschullehrerausbildung Südsudan, Fortfüh...",2024-01-22T00:00:00Z,"[2017-01-01T00:00:00Z, 2017-01-01T00:00:00Z, 2...",[SS],[Juba],[Bundesministerium für wirtschaftliche Zusamme...,"[de, en]","[de, en, de, en]",[Bundesministerium für wirtschaftliche Zusamme...,NaN
5,[31130],DE-1-201701127-2712,[SEWOH - Projekt zur Ernährungssicherung von b...,"[https://www.bmz.de/de/laender/suedsudan, http...",DE-1,[1],"[1, 2, 3, 4]",3,[SEWOH - Projekt zur Ernährungssicherung von b...,2024-01-22T00:00:00Z,"[2018-01-01T00:00:00Z, 2018-01-01T00:00:00Z, 2...",[SS],[Juba],[Bundesministerium für wirtschaftliche Zusamme...,"[de, en]","[de, en, de, en]",[Bundesministerium für wirtschaftliche Zusamme...,NaN
6,[14030],DE-1-201706019-2492,[Verbesserte Grundversorgung hinsichtlich WASH...,"[https://www.bmz.de/de/laender/suedsudan, http...",DE-1,[1],"[1, 2, 3, 4]",4,[Verbesserte Grundversorgung hinsichtlich WASH...,2024-01-22T00:00:00Z,"[2017-09-01T00:00:00Z, 2017-09-01T00:00:00Z, 2...",[SS],[Juba],[Bundesministerium für wirtschaftliche Zusamme...,"[de, en]","[de, en, de, en]",[Bundesministerium für wirtschaftliche Zusamme...,NaN
7,[12230],DE-1-201706019-2695,[Aufbau der Basisgesundheitsversorgung in Make...,"[https://www.bmz.de/de/laender/suedsudan, http...",DE-1,[1],"[1, 2, 3, 4]",4,[Aufbau der Basisgesundheitsversorgung in Make...,2024-01-22T00:00:00Z,"[2017-10-01T00:00:00Z, 2017-10-01T00:00:00Z, 2...",[SS],[Juba],[Bundesministerium für wirtschaftliche Zusamme...,"[de, en]","[de, en, de, en]",[Bundesministerium für wirtschaftliche Zusamme...,NaN
8,[12220],DE-1-201706019-2836,[Berufsausbildung und Einkommensmöglichkeiten ...,"[https://www.bmz.de/de/laender/suedsudan, http...",DE-1,[1],"[1, 2, 3, 4]",4,[Berufsausbildung und Einkommensmöglichkeiten 

In [143]:
print(f"Entries: {len(df)}")
df.reporting_org_ref.value_counts()

Entries: 48


XM-DAC-5-52    48
Name: reporting_org_ref, dtype: int64

In [144]:
# create new empty df to fill with transformed data

trans_df = pd.DataFrame()

## **Feature Transformation & Engineering**

### IATI ID

In [145]:
trans_df["iati_id"] = df["iati_identifier"].values

trans_df.head(2)

,iati_id
0,XM-DAC-5-52-197230105
1,XM-DAC-5-52-197030380


### Title

In [146]:
################
# Add en title #
################

pbar = tqdm(total=len(df))
trans_df["title_en"] = "NaN"

for index, row in df.iterrows():
    title_row = row['title_narrative']
    try:
        if 'title_narrative_xml_lang' in df.columns:
            lang_list = row['title_narrative_xml_lang']

            # nan in pandas is type float
            # check if nan and if yes take first entry in lang
            if isinstance(lang_list, float):
                if isinstance(title_row, float):
                    trans_df["title_en"][index] = "NaN"
                else:
                    trans_df["title_en"][index] = title_row[0]
            elif len(lang_list) == len(title_row):
                for j in range(0, len(lang_list)):
                    if "en" or "EN" in lang_list:
                        if lang_list[j].lower() == "en":
                            title = title_row[j]
                            trans_df["title_en"][index] = title
                    else:
                        trans_df["title_en"][index] = "NaN"
            else:
                pass
        else:
            trans_df["title_en"][index] = row['title_narrative'][0]
    except:
        print(f"Error: Index: {index}, Row: {lang_list}, {title_row}")
    
    pbar.update(1)

pbar.close()

trans_df.head(2)

  0%|          | 0/48 [00:00<?, ?it/s]

,iati_id,title_en
0,XM-DAC-5-52-197230105,AFR100 Regional Coordinators - Phase 2
1,XM-DAC-5-52-197030380,AFR 100 Regional Coordinators


In [147]:
###################
# Add other title #
###################

pbar = tqdm(total=len(df))

trans_df["title_other"] = "NaN"

for index, row in df.iterrows():
    try:
        if 'title_narrative_xml_lang' in df.columns:
            lang_list = row['title_narrative_xml_lang']

            # every title which is has no lang attribute is classified as English and therefore not in other
            if isinstance(lang_list, float):
                trans_df["title_other"][index] = "NaN"
            elif len(lang_list) == len(title_row):
                for j in range(0, len(lang_list)):
                    if lang_list[j].lower() != "en":
                        title = row['title_narrative'][j]
                        if trans_df["title_other"][index] == "NaN":
                            trans_df["title_other"][index] = title
                        else:
                            trans_df["title_other"][index] = f"{trans_df['title_other'][index]}; {title}"
            else:
                trans_df["title_other"][index] = title
            
            pbar.update(1)

        else:
            trans_df["title_other"][index] = "NaN"
        
    except Exception as e:
        print(f"Error: Index: {index} \n Row: {row} \n Exception: {e}")

pbar.close()

trans_df.head(200)

  0%|          | 0/48 [00:00<?, ?it/s]

,iati_id,title_en,title_other
0,XM-DAC-5-52-197230105,AFR100 Regional Coordinators - Phase 2,NaN
1,XM-DAC-5-52-197030380,AFR 100 Regional Coordinators,NaN
2,XM-DAC-5-52-196930325,CSSF Libya - Decentralisation,NaN
3,XM-DAC-5-52-197230220,Ethiopian Forest Coffee Value Chain (EWCP 2.0),NaN
4,XM-DAC-5-52-196930572,EU EOM Kosovo 2019,NaN
5,XM-DAC-5-52-197030109,PO 8419 Strengthening Malawi's education system,NaN
6,XM-DAC-5-52-196930630,Gender Based Violence (GBV),NaN
7,XM-DAC-5-52-197030448,VET4All - Vocational and Technical Education f...,NaN
8,XM-DAC-5-52-197130107,Support to the Implementation of the TVSDC Act...,NaN
9,XM-DAC-5-52-197130248,ALPHA 3 – AT stratégie d’alphabétisation,NaN


### Organization

In [148]:
trans_df['organization'] = df['reporting_org_narrative'].apply(lambda x: x[0])

print(trans_df.organization.value_counts())
trans_df.head(2)

Germany GIZ Non BMZ    48
Name: organization, dtype: int64


,iati_id,title_en,title_other,organization
0,XM-DAC-5-52-197230105,AFR100 Regional Coordinators - Phase 2,NaN,Germany GIZ Non BMZ
1,XM-DAC-5-52-197030380,AFR 100 Regional Coordinators,NaN,Germany GIZ Non BMZ


### Country

In [149]:
trans_df["country"] = df["recipient_country_code"]
trans_df.head(5)

,iati_id,title_en,title_other,organization,country
0,XM-DAC-5-52-197230105,AFR100 Regional Coordinators - Phase 2,NaN,Germany GIZ Non BMZ,NaN
1,XM-DAC-5-52-197030380,AFR 100 Regional Coordinators,NaN,Germany GIZ Non BMZ,NaN
2,XM-DAC-5-52-196930325,CSSF Libya - Decentralisation,NaN,Germany GIZ Non BMZ,[LY]
3,XM-DAC-5-52-197230220,Ethiopian Forest Coffee Value Chain (EWCP 2.0),NaN,Germany GIZ Non BMZ,[ET]
4,XM-DAC-5-52-196930572,EU EOM Kosovo 2019,NaN,Germany GIZ Non BMZ,[XK]


### Region

In [150]:
trans_df['region'] = df['recipient_region_code']
trans_df.head(5)

,iati_id,title_en,title_other,organization,country,region
0,XM-DAC-5-52-197230105,AFR100 Regional Coordinators - Phase 2,NaN,Germany GIZ Non BMZ,NaN,[289]
1,XM-DAC-5-52-197030380,AFR 100 Regional Coordinators,NaN,Germany GIZ Non BMZ,NaN,[289]
2,XM-DAC-5-52-196930325,CSSF Libya - Decentralisation,NaN,Germany GIZ Non BMZ,[LY],NaN
3,XM-DAC-5-52-197230220,Ethiopian Forest Coffee Value Chain (EWCP 2.0),NaN,Germany GIZ Non BMZ,[ET],NaN
4,XM-DAC-5-52-196930572,EU EOM Kosovo 2019,NaN,Germany GIZ Non BMZ,[XK],NaN


### Location

In [151]:
try: 
    if 'title_narrative_xml_lang' in df.columns:
        trans_df['location'] = df['location_name_narrative']
    else:
        trans_df['location'] = "NaN"
except:
        trans_df['location'] = "NaN"    
trans_df.head(5)

,iati_id,title_en,title_other,organization,country,region,location
0,XM-DAC-5-52-197230105,AFR100 Regional Coordinators - Phase 2,NaN,Germany GIZ Non BMZ,NaN,[289],NaN
1,XM-DAC-5-52-197030380,AFR 100 Regional Coordinators,NaN,Germany GIZ Non BMZ,NaN,[289],NaN
2,XM-DAC-5-52-196930325,CSSF Libya - Decentralisation,NaN,Germany GIZ Non BMZ,[LY],NaN,NaN
3,XM-DAC-5-52-197230220,Ethiopian Forest Coffee Value Chain (EWCP 2.0),NaN,Germany GIZ Non BMZ,[ET],NaN,NaN
4,XM-DAC-5-52-196930572,EU EOM Kosovo 2019,NaN,Germany GIZ Non BMZ,[XK],NaN,NaN


### Descriptions

descr1, descri2, ... can have overlap. Doppelte Beschriebungen entfernen! Es gibt lib um overlap heruaszufinden!

In [152]:
#################################
# Description english and other #
#################################

trans_df["description_en"] = "NaN"
trans_df["description_other"] = "NaN"

for index, row in df.iterrows():

    try:
        if 'description_narrative_xml_lang' in df.columns:
            descr_list = row['description_narrative_xml_lang']
            descr_row = row['description_narrative']

            # nan in pandas is type float
            # check if nan and if yes take first entry in descr
            if isinstance(descr_list, float):
                if isinstance(descr_row, float):
                    trans_df["description_en"][index] = "NaN"
                else:
                    trans_df["description_en"][index] = descr_row[0]
            else:
                # iterate throug description list
                for j in range(0, len(descr_list)):
                    # if description english
                    if descr_list[j].lower() == "en":
                        if type(descr_row) == float:
                            descr = "NaN"
                        else:
                            descr = descr_row[j]
                        if trans_df["description_en"][index] == "NaN":
                            trans_df["description_en"][index] = descr
                        else:
                            trans_df["description_en"][index] = f"{trans_df['description_en'][index]}; {descr}"
                    else:
                        if type(descr_row) == float:
                            descr = "NaN"
                        else:
                            descr = descr_row[j]
                        if trans_df["description_other"][index] == "NaN":
                            trans_df["description_other"][index] = descr
                        else:
                            trans_df["description_other"][index] = f"{trans_df['description_other'][index]}; {descr}"
        else:
            descr_str = ""
            for d in row['description_narrative']:
                descr_str += f"{d}; "
            trans_df["description_en"][index] = descr_str

    except:
        print(f"Error: Index: {index} \n Row: {row}")
        
trans_df.head(5)

,iati_id,title_en,title_other,organization,country,region,location,description_en,description_other
0,XM-DAC-5-52-197230105,AFR100 Regional Coordinators - Phase 2,NaN,Germany GIZ Non BMZ,NaN,[289],NaN,AFR100 Regional Coordinators - Phase 2; See do...,NaN
1,XM-DAC-5-52-197030380,AFR 100 Regional Coordinators,NaN,Germany GIZ Non BMZ,NaN,[289],NaN,AFR 100 Regional Coordinators; See document links,NaN
2,XM-DAC-5-52-196930325,CSSF Libya - Decentralisation,NaN,Germany GIZ Non BMZ,[LY],NaN,NaN,CSSF Libya - Decentralisation; See document links,NaN
3,XM-DAC-5-52-197230220,Ethiopian Forest Coffee Value Chain (EWCP 2.0),NaN,Germany GIZ Non BMZ,[ET],NaN,NaN,Ethiopian Forest Coffee Value Chain (EWCP 2.0)...,NaN
4,XM-DAC-5-52-196930572,EU EOM Kosovo 2019,NaN,Germany GIZ Non BMZ,[XK],NaN,NaN,EU EOM Kosovo 2019; See document links,NaN


### Status

In [153]:
# https://iatistandard.org/en/iati-standard/203/codelists/activitystatus/
activity_status = {
    1: "Pipeline/identification",
    2: "Implementation",
    3: "Finalisation",
    4: "Closed",
    5: "Cancelled",
    6: "Suspended"
}

trans_df["status"] = df.activity_status_code
trans_df['status'] = trans_df['status'].replace(activity_status)

trans_df.head(2)
    

,iati_id,title_en,title_other,organization,country,region,location,description_en,description_other,status
0,XM-DAC-5-52-197230105,AFR100 Regional Coordinators - Phase 2,NaN,Germany GIZ Non BMZ,NaN,[289],NaN,AFR100 Regional Coordinators - Phase 2; See do...,NaN,Finalisation
1,XM-DAC-5-52-197030380,AFR 100 Regional Coordinators,NaN,Germany GIZ Non BMZ,NaN,[289],NaN,AFR 100 Regional Coordinators; See document links,NaN,Closed


### Date

In [154]:
#############
# Date Type #
#############

# One Hot
# 1 -> Yes
# 0 -> No

# Codes:
# 1 Planned start
# 2 Actual start
# 3 Planned end
# 4 Actual end

trans_df["planned_start"] = "NaN"
trans_df["actual_start"] = "NaN"
trans_df["planned_end"] = "NaN"
trans_df["actual_end"] = "NaN"

date_types = {
    1: "planned_start",
    2: "actual_start",
    3: "planned_end",
    4: "actual_end"
}

for index, row in df.iterrows():
    dtype_list = row["activity_date_type"]
    iso_date_list = row["activity_date_iso_date"]

    combined_list = list(zip(dtype_list, iso_date_list))

    # replace nums with column names from date_types
    combined_list = [(date_types[int(t[0])], t[1]) for t in combined_list]

    for i in combined_list:
        trans_df[i[0]] = i[1]

trans_df.head(1800)


,iati_id,title_en,title_other,organization,country,region,location,description_en,description_other,status,planned_start,actual_start,planned_end,actual_end
0,XM-DAC-5-52-197230105,AFR100 Regional Coordinators - Phase 2,NaN,Germany GIZ Non BMZ,NaN,[289],NaN,AFR100 Regional Coordinators - Phase 2; See do...,NaN,Finalisation,2023-09-13T00:00:00Z,NaN,2024-09-12T00:00:00Z,NaN
1,XM-DAC-5-52-197030380,AFR 100 Regional Coordinators,NaN,Germany GIZ Non BMZ,NaN,[289],NaN,AFR 100 Regional Coordinators; See document links,NaN,Closed,2023-09-13T00:00:00Z,NaN,2024-09-12T00:00:00Z,NaN
2,XM-DAC-5-52-196930325,CSSF Libya - Decentralisation,NaN,Germany GIZ Non BMZ,[LY],NaN,NaN,CSSF Libya - Decentralisation; See document links,NaN,Finalisation,2023-09-13T00:00:00Z,NaN,2024-09-12T00:00:00Z,NaN
3,XM-DAC-5-52-197230220,Ethiopian Forest Coffee Value Chain (EWCP 2.0),NaN,Germany GIZ Non BMZ,[ET],NaN,NaN,Ethiopian Forest Coffee Value Chain (EWCP 2.0)...,NaN,Finalisation,2023-09-13T00:00:00Z,NaN,2024-09-12T00:00:00Z,NaN
4,XM-DAC-5-52-196930572,EU EOM Kosovo 2019,NaN,Germany GIZ Non BMZ,[XK],NaN,NaN,EU EOM Kosovo 2019; See document links,NaN,Finalisation,2023-09-13T00:00:00Z,NaN,2024-09-12T00:00:00Z,NaN
5,XM-DAC-5-52-197030109,PO 8419 Strengthening Malawi's education system,NaN,Germany GIZ Non BMZ,[MW],NaN,NaN,PO 8419 Strengthening Malawi's education syste...,NaN,Closed,2023-09-13T00:00:00Z,NaN,2024-09-12T00:00:00Z,NaN
6,XM-DAC-5-52-196930630,Gender Based Violence (GBV),NaN,Germany GIZ Non BMZ,[ZM],NaN,NaN,Gender Based Violence (GBV); See document links,NaN,Finalisation,2023-09-13T00:00:00Z,NaN,2024-09-12T00:00:00Z,NaN
7,XM-DAC-5-52-197030448,VET4All - Vocational and Technical Education f...,NaN,Germany GIZ Non BMZ,[LB],NaN,NaN,VET4All - Vocational and Technical Education f...,NaN,Finalisation,2023-09-13T00:00:00Z,NaN,2024-09-12T00:00:00Z,NaN
8,XM-DAC-5-52-197130107,Support to the Implementation of the TVSDC Act...,NaN,Germany GIZ Non BMZ,[JO],NaN,NaN,Support to the Implementation of the TVSDC Act...,NaN,Finalisation,2023-09-13T00:00:00Z,NaN,2024-09-12T00:00:00Z,NaN
9,XM-DAC-5-52-197130248,ALPHA 3 – AT stratégie d’alphabétisation,NaN,Germany GIZ Non BMZ,[MA],NaN,NaN,ALPHA 3 – AT stratégie d’alphabétisation; See ...,NaN,Finalisation,2023-09-13T00:00:00Z,NaN,2024-09-12T00:00:00Z,NaN


### Last Update

In [155]:
trans_df['last_update'] = df['last_updated_datetime']
trans_df.head(2)

,iati_id,title_en,title_other,organization,country,region,location,description_en,description_other,status,planned_start,actual_start,planned_end,actual_end,last_update
0,XM-DAC-5-52-197230105,AFR100 Regional Coordinators - Phase 2,NaN,Germany GIZ Non BMZ,NaN,[289],NaN,AFR100 Regional Coordinators - Phase 2; See do...,NaN,Finalisation,2023-09-13T00:00:00Z,NaN,2024-09-12T00:00:00Z,NaN,2023-07-03T09:56:30Z
1,XM-DAC-5-52-197030380,AFR 100 Regional Coordinators,NaN,Germany GIZ Non BMZ,NaN,[289],NaN,AFR 100 Regional Coordinators; See document links,NaN,Closed,2023-09-13T00:00:00Z,NaN,2024-09-12T00:00:00Z,NaN,2022-08-31T09:00:22Z


### Sector Codes (CRS & SGDs)

In [156]:
sector_codes = {
    1: "OECD DAC CRS Purpose Codes (5 digit)",
    2: "OECD DAC CRS Purpose Codes (3 digit)",
    3: "Classification of the Functions of Government (UN)",
    4: "Statistical classification of economic activities in the European Community",
    5: "National Taxonomy for Exempt Entities (USA)",
    6: "AidData",
    7: "SDG Goal",
    8: "SDG Target",
    9: "SDG Indicator",
    10: "Humanitarian Global Clusters (Inter-Agency Standing Committee)",
    11: "North American Industry Classification System (NAICS)",
    12: "UN System Function",
    99: "Reporting Organisation", # The sector reported corresponds to a sector vocabulary maintained by the reporting organisation for this activity
    98: "Reporting Organisation 2" # The sector reported corresponds to a sector vocabulary maintained by the reporting organisation for this activity (if they are referencing more than one)
}

In [157]:
crs5_df = pd.read_csv("../../src/crs5_codes.csv")
crs5_df.head(2)

,code,name,description,language,category,category-name,category-description
0,11110,Education policy and administrative management,"Education sector policy, planning and programm...",en,111,"Education, Level Unspecified",The codes in this category are to be used only...
1,11120,Education facilities and training,"Educational buildings, equipment, materials; s...",en,111,"Education, Level Unspecified",The codes in this category are to be used only...


In [158]:
crs3_df = pd.read_csv("../../src/crs3_codes.csv")
crs3_df.head(2)

,code,name,description,language,category,category-name,category-description
0,111,"Education, Level Unspecified",The codes in this category are to be used only...,en,NaN,NaN,NaN
1,112,Basic Education,NaN,en,NaN,NaN,NaN


#### Get CRS5 and CRS3 Tags

other tags are not given by organizations in IATI data; need other ways to find out these tags

In [159]:
# to process different variants of codes (crs3, crs5 etc.)

def process_codes(combined_list, translation_df, code_index):
    code_text = ""
    codes_nums = ""
    if any(item[0] == code_index for item in combined_list):
        for i in combined_list:
            if i[0] == code_index:
                translation = translation_df.loc[translation_df['code'] == int(i[1]), 'name'].values[0]
                code_text += f"{translation}; "
                codes_nums += f"{i[1]}; "
        return code_text, codes_nums
    else:
        return "NaN", "NaN"
    
def derive_crs3(combined_list, translation_df, code_index="1"):
    code_text = ""
    codes_nums = ""
    if any(item[0] == code_index for item in combined_list):
        for i in combined_list:
            if i[0] == code_index:
                translation = translation_df.loc[translation_df['code'] == int(i[1][:3]), 'name'].values[0]
                code_text += f"{translation}; "
                codes_nums += f"{i[1][:3]}; "
        return code_text, codes_nums
    else:
        return "NaN", "NaN"

In [160]:
###############
# ECTRACT CRS #
###############
trans_df["crs_5_code"] = "NaN"
trans_df["crs_5_name"] = "NaN"

# Most Project dont have information on crs3 -> crs3 derived from crs5 tags
trans_df["crs_3_code"] = "NaN"
trans_df["crs_3_name"] = "NaN"


for index, row in df.iterrows():
    crs_voc_list = row['sector_vocabulary']
    crs_code_list = row['sector_code']

    if type(crs_voc_list) == float:
        pass
    else:
        try:
            combined_list = list(zip(crs_voc_list, crs_code_list))

            # CRS 5
            crs5_str, crs5_codes = process_codes(combined_list, crs5_df, "1")
            trans_df["crs_5_code"][index] = crs5_codes
            trans_df["crs_5_name"][index] = crs5_str
            
            # CRS 3
            crs3_str, crs3_codes = derive_crs3(combined_list, crs3_df)
            trans_df["crs_3_code"][index] = crs3_codes
            trans_df["crs_3_name"][index] = crs3_str

        except:
            print(f"Error on Index {index}, {crs_code_list}")
            pass

trans_df.head(5)

,iati_id,title_en,title_other,organization,country,region,location,description_en,description_other,status,planned_start,actual_start,planned_end,actual_end,last_update,crs_5_code,crs_5_name,crs_3_code,crs_3_name
0,XM-DAC-5-52-197230105,AFR100 Regional Coordinators - Phase 2,NaN,Germany GIZ Non BMZ,NaN,[289],NaN,AFR100 Regional Coordinators - Phase 2; See do...,NaN,Finalisation,2023-09-13T00:00:00Z,NaN,2024-09-12T00:00:00Z,NaN,2023-07-03T09:56:30Z,41030;,Biodiversity;,410;,General Environment Protection;
1,XM-DAC-5-52-197030380,AFR 100 Regional Coordinators,NaN,Germany GIZ Non BMZ,NaN,[289],NaN,AFR 100 Regional Coordinators; See document links,NaN,Closed,2023-09-13T00:00:00Z,NaN,2024-09-12T00:00:00Z,NaN,2022-08-31T09:00:22Z,41030;,Biodiversity;,410;,General Environment Protection;
2,XM-DAC-5-52-196930325,CSSF Libya - Decentralisation,NaN,Germany GIZ Non BMZ,[LY],NaN,NaN,CSSF Libya - Decentralisation; See document links,NaN,Finalisation,2023-09-13T00:00:00Z,NaN,2024-09-12T00:00:00Z,NaN,2022-08-22T17:09:10Z,15112;,Decentralisation and support to subnational go...,151;,Government & Civil Society-general;
3,XM-DAC-5-52-197230220,Ethiopian Forest Coffee Value Chain (EWCP 2.0),NaN,Germany GIZ Non BMZ,[ET],NaN,NaN,Ethiopian Forest Coffee Value Chain (EWCP 2.0)...,NaN,Finalisation,2023-09-13T00:00:00Z,NaN,2024-09-12T00:00:00Z,NaN,2023-07-03T09:56:05Z,25010;,Business policy and administration;,250;,Business & Other Services;
4,XM-DAC-5-52-196930572,EU EOM Kosovo 2019,NaN,Germany GIZ Non BMZ,[XK],NaN,NaN,EU EOM Kosovo 2019; See document links,NaN,Finalisation,2023-09-13T00:00:00Z,NaN,2024-09-12T00:00:00Z,NaN,2022-11-14T16:05:14Z,15151;,Elections;,151;,Government & Civil Society-general;


#### Derive crs3 tags from crs5 tags

It is possible to derive CRS3 tags from CRS5 tags by looking at the first 3 characters.

### Documents

In [161]:
df.document_link_url.value_counts()

trans_df['docs'] = df['document_link_url']
trans_df.head(2)

,iati_id,title_en,title_other,organization,country,region,location,description_en,description_other,status,planned_start,actual_start,planned_end,actual_end,last_update,crs_5_code,crs_5_name,crs_3_code,crs_3_name,docs
0,XM-DAC-5-52-197230105,AFR100 Regional Coordinators - Phase 2,NaN,Germany GIZ Non BMZ,NaN,[289],NaN,AFR100 Regional Coordinators - Phase 2; See do...,NaN,Finalisation,2023-09-13T00:00:00Z,NaN,2024-09-12T00:00:00Z,NaN,2023-07-03T09:56:30Z,41030;,Biodiversity;,410;,General Environment Protection;,[https://www.giz.de/projektdaten/projects.acti...
1,XM-DAC-5-52-197030380,AFR 100 Regional Coordinators,NaN,Germany GIZ Non BMZ,NaN,[289],NaN,AFR 100 Regional Coordinators; See document links,NaN,Closed,2023-09-13T00:00:00Z,NaN,2024-09-12T00:00:00Z,NaN,2022-08-31T09:00:22Z,41030;,Biodiversity;,410;,General Environment Protection;,[https://www.giz.de/projektdaten/projects.acti...


## **Save final DF as CSV**

In [162]:
trans_df.head()

,iati_id,title_en,title_other,organization,country,region,location,description_en,description_other,status,planned_start,actual_start,planned_end,actual_end,last_update,crs_5_code,crs_5_name,crs_3_code,crs_3_name,docs
0,XM-DAC-5-52-197230105,AFR100 Regional Coordinators - Phase 2,NaN,Germany GIZ Non BMZ,NaN,[289],NaN,AFR100 Regional Coordinators - Phase 2; See do...,NaN,Finalisation,2023-09-13T00:00:00Z,NaN,2024-09-12T00:00:00Z,NaN,2023-07-03T09:56:30Z,41030;,Biodiversity;,410;,General Environment Protection;,[https://www.giz.de/projektdaten/projects.acti...
1,XM-DAC-5-52-197030380,AFR 100 Regional Coordinators,NaN,Germany GIZ Non BMZ,NaN,[289],NaN,AFR 100 Regional Coordinators; See document links,NaN,Closed,2023-09-13T00:00:00Z,NaN,2024-09-12T00:00:00Z,NaN,2022-08-31T09:00:22Z,41030;,Biodiversity;,410;,General Environment Protection;,[https://www.giz.de/projektdaten/projects.acti...
2,XM-DAC-5-52-196930325,CSSF Libya - Decentralisation,NaN,Germany GIZ Non BMZ,[LY],NaN,NaN,CSSF Libya - Decentralisation; See document links,NaN,Finalisation,2023-09-13T00:00:00Z,NaN,2024-09-12T00:00:00Z,NaN,2022-08-22T17:09:10Z,15112;,Decentralisation and support to subnational go...,151;,Government & Civil Society-general;,[https://www.giz.de/projektdaten/projects.acti...
3,XM-DAC-5-52-197230220,Ethiopian Forest Coffee Value Chain (EWCP 2.0),NaN,Germany GIZ Non BMZ,[ET],NaN,NaN,Ethiopian Forest Coffee Value Chain (EWCP 2.0)...,NaN,Finalisation,2023-09-13T00:00:00Z,NaN,2024-09-12T00:00:00Z,NaN,2023-07-03T09:56:05Z,25010;,Business policy and administration;,250;,Business & Other Services;,[https://www.giz.de/projektdaten/projects.acti...
4,XM-DAC-5-52-196930572,EU EOM Kosovo 2019,NaN,Germany GIZ Non BMZ,[XK],NaN,NaN,EU EOM Kosovo 2019; See document links,NaN,Finalisation,2023-09-13T00:00:00Z,NaN,2024-09-12T00:00:00Z,NaN,2022-11-14T16:05:14Z,15151;,Elections;,151;,Government & Civil Society-general;,[https://www.giz.de/projektdaten/projects.acti...


In [163]:
trans_df.to_csv(output_file, index=False) 